# Credit Risk Classification

Credit risk poses a classification problem that’s inherently imbalanced. This is because healthy loans easily outnumber risky loans. In this Challenge, you’ll use various techniques to train and evaluate models with imbalanced classes. You’ll use a dataset of historical lending activity from a peer-to-peer lending services company to build a model that can identify the creditworthiness of borrowers.

## Instructions:

This challenge consists of the following subsections:

* Split the Data into Training and Testing Sets

* Create a Logistic Regression Model with the Original Data

* Predict a Logistic Regression Model with Resampled Training Data 

### Split the Data into Training and Testing Sets

Open the starter code notebook and then use it to complete the following steps.

1. Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

2. Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

    > **Note** A value of `0` in the “loan_status” column means that the loan is healthy. A value of `1` means that the loan has a high risk of defaulting.  

3. Check the balance of the labels variable (`y`) by using the `value_counts` function.

4. Split the data into training and testing datasets by using `train_test_split`.

### Create a Logistic Regression Model with the Original Data

Employ your knowledge of logistic regression to complete the following steps:

1. Fit a logistic regression model by using the training data (`X_train` and `y_train`).

2. Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

3. Evaluate the model’s performance by doing the following:

    * Calculate the accuracy score of the model.

    * Generate a confusion matrix.

    * Print the classification report.

4. Answer the following question: How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

### Predict a Logistic Regression Model with Resampled Training Data

Did you notice the small number of high-risk loan labels? Perhaps, a model that uses resampled data will perform better. You’ll thus resample the training data and then reevaluate the model. Specifically, you’ll use `RandomOverSampler`.

To do so, complete the following steps:

1. Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

2. Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

3. Evaluate the model’s performance by doing the following:

    * Calculate the accuracy score of the model.

    * Generate a confusion matrix.

    * Print the classification report.
    
4. Answer the following question: How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

### Write a Credit Risk Analysis Report

For this section, you’ll write a brief report that includes a summary and an analysis of the performance of both machine learning models that you used in this challenge. You should write this report as the `README.md` file included in your GitHub repository.

Structure your report by using the report template that `Starter_Code.zip` includes, and make sure that it contains the following:

1. An overview of the analysis: Explain the purpose of this analysis.


2. The results: Using bulleted lists, describe the balanced accuracy scores and the precision and recall scores of both machine learning models.

3. A summary: Summarize the results from the machine learning models. Compare the two versions of the dataset predictions. Include your recommendation for the model to use, if any, on the original vs. the resampled data. If you don’t recommend either model, justify your reasoning.

In [ ]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path

# SciKit-Learn modules   See https://scikit-learn.org/stable/install.html for dependencies and installation
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Imbalanced-Learn modules  See https://pypi.org/project/imbalanced-learn/ for dependencies and installation
from imblearn.metrics import classification_report_imbalanced

import warnings
warnings.filterwarnings('ignore')

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [ ]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df_input_data = pd.read_csv(                  # Read the specified csv file into the dataframe
    Path("../../Resources/lending_data.csv")  #  using a normalized version of the path and filename for the operating system on which the code is running
)

# Review the DataFrame
display( "*** Head ***", df_input_data.head() );  # Add a heading and show the first few rows of the dataframe
display( "*** Checking for Nulls ***", df_input_data.isna().sum() );  # Show the number of nulls in each column out of interest to determine the quality of the data

### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [ ]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df_input_data["loan_status"]  # Create the y (labels set) from the loan_status column of the dataset

# Separate the X variable, the features
X = df_input_data.drop(columns=["loan_status"]).copy() # Create the X (features set) by copying the DF and dropping the Labels set

In [ ]:
# Review the y variable Series
y[:5] # Preview 5 rows of y

In [ ]:
# Review the X variable DataFrame
X.head() # Preview 5 rows of X

### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [ ]:
# Check the balance of our target values
y.value_counts() # Show a count of each different value in the labels variable (y)

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [ ]:
# Split data into into random Train and Test subsets.
X_train, X_test, y_train, y_test = train_test_split(X, y, # Split the X and y input data into random Train and Test subsets.
                                                    random_state=1) # Set random_state to ensure results are the same when repeating execution

In [ ]:
display("*** X_train Info ***", X_train.info()) # Show information about the data subset

In [ ]:
display("*** X_test Info ***", X_test.info()) # Show information about the data subset

In [ ]:
display("*** y_train Info ***", y_train.info()) # Show information about the data subset

In [ ]:
display("*** y_test Info ***", y_test.info()) # Show information about the data subset

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [ ]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(  # Instantiate Logistic Regression model
    solver="lbfgs",                              # ...setting solver to lbfgs for clarity even though that's the default 
    random_state=1                               # ...with a random_state seed of 1 to set randomness within the same pattern
)

# Fit the model using training data
logistic_regression_model.fit( X_train, y_train ) # Fit the regression model using the training data

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [ ]:
# Make a prediction using the testing data
predictions = logistic_regression_model.predict(X_test)              

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [ ]:
# Print the balanced_accuracy score of the model
display("*** Balanced Accuracy Score ***", balanced_accuracy_score(y_test, predictions) ) # Display a banner and the balanced accuracy score

In [ ]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, predictions) # Generate a confusion matrix using the y_test and predictions

In [ ]:
# Print the classification report for the model
print(classification_report(y_test, predictions)) # Print (works better than display) the classification report

### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** 
For the healthy loan group the F1-Score was 100% as a result of 100% Precision score (proportion of true positives) of the and a 99% Recall score (of all the actual healthy loans how many it predicted correctly). The large dataset (18756 samples) would likely have contributed to this accuracy. The balanced accuracy score (i.e. the average of recall obtained on each class) resulted in 95%. The overall accuracy of the model scored 99% which is very good.

In contrast, the high-risk loan group had a lower F1-Score of 88%, driven by a Precision score of 85% and a Recall score of 91%. With only 619 records as reference data for high-risk loans, there is a significant imbalance in the dataset. Given the use case of needing to correctly detect high-risk loans, and that the dataset is likely to generally be largely unbalanced, it is worth applying other analysis tools to determine if there is scope for improvement.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [ ]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
 # Assign a random_state parameter of 1 to the model
ros = RandomOverSampler(random_state=1) # Instantiate the random oversampler

# Fit the original training data to the random_oversampler model
X_oversampled, y_oversampled = ros.fit_resample(X_train, y_train) # Fit the original training data to the random_oversampler model

In [ ]:
# Count the distinct values of the resampled labels data
y_oversampled.value_counts()  # Output the count of unique values

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [ ]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
resampled_lrm = LogisticRegression(  # Instantiate Logistic Regression model for the resampled data
    solver="lbfgs",                  # ...setting solver to lbfgs for clarity even though that's the default 
    random_state=1                   # ...with a random_state seed of 1 to set randomness within the same pattern
)

# Fit the model using the resampled training data
resampled_lrm.fit( X_oversampled, y_oversampled ) # Fit the regression model using the training data

# Make a prediction using the testing data
resampled_predictions = resampled_lrm.predict(X_test)  # predict with the oversample model and test data

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [ ]:
# Print the balanced_accuracy score of the model 
display("*** Balanced Accuracy Score of Oversampled Model ***", balanced_accuracy_score(y_test, resampled_predictions) ) # Display a banner and the balanced accuracy score on the resampled model

In [ ]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, resampled_predictions) # Generate a confusion matrix using the y_test and oversampled predictions

In [ ]:
# Print the classification report for the model
print(classification_report(y_test, resampled_predictions)) # Print (works better than display) the classification report

### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model (LRM) fit with oversampled data showed the same results in terms of F1-Score, Accuracy, Precision and Recall as the LRM using the original data. However, the balanced accuracy score improved from 95% to 99%.

With the LRM, the healthy loan group had an F1-Score of 100%, Accuracy of 99%, Precision score of 100% and Recall score of 99%; while the high-risk loan group had an F1-Score of 88%, Precision score of 85% and a Recall score of 91%.

In [ ]:
# Re-print the classification reports to make it easier to compare
print(f"Classifiction Report - Original Data")
print(classification_report(y_test, predictions)) # Show the original predictions
print("---------")
print(f"Classifiction Report - Oversampled Data")
print(classification_report(y_test, resampled_predictions))  # Show the resampled / oversampled predictions

---

# Further analysis on the dataset to assist in completing the report

In [ ]:
# Presenting averages by loan status (0=healthy loan group, 1=high-risk loan group) to assist in further analysis, in particular if there is any relationship
# with the derogatory marks (negative items such as a late payment or foreclosures against the account) 
df_summary_averages = df_input_data.groupby(by=df_input_data["loan_status"]).mean() # Capture the averages from the all columns
display("*** Dataset Averages by Loan Status ***", df_summary_averages)